In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import sqlite3
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

db_path = 'data.db'
connection = sqlite3.connect(db_path)

queryplayers = "SELECT * FROM players"
players = pd.read_sql_query(queryplayers, connection)

queryTeams = "SELECT * FROM teams"
teams = pd.read_sql_query(queryTeams, connection)

queryplayers_teams = "SELECT * FROM players_teams"
players_teams = pd.read_sql_query(queryplayers_teams, connection)

queryawards = "SELECT * FROM awards_players"
awards_players = pd.read_sql_query(queryawards, connection)

querycoaches = "SELECT * FROM coaches"
coaches = pd.read_sql_query(querycoaches, connection)

queryseries_post = "SELECT * FROM coaches"
series_post = pd.read_sql_query(queryseries_post, connection)

queryteams_post = "SELECT * FROM coaches"
teams_post = pd.read_sql_query(queryteams_post, connection)


### Awards_players


- corrected name of a prize
- drop lgID
- calculate the number of prizes that each player has (having in consideration the year)
- drop award (name)

In [ ]:
awards_players['award'] = awards_players['award'].replace("Kim Perrot Sportsmanship", "Kim Perrot Sportsmanship Award")
awards_players = awards_players.drop(columns=['lgID'], axis=1)

award_weights = {
    "All-Star Game Most Valuable Player": 2,
    "Coach of the Year": 3,
    "Defensive Player of the Year": 4,
    "Kim Perrot Sportsmanship Award": 1,
    "Most Improved Player": 2,
    "Most Valuable Player": 5,
    "Rookie of the Year": 3,
    "Sixth Woman of the Year": 2,
    "WNBA Finals Most Valuable Player": 4,
    "WNBA All-Decade Team": 1,
    "WNBA All Decade Team Honorable Mention": 1,
}

# Sort the DataFrame by playerID and year
awards_players.sort_values(by=["playerID", "year"], inplace=True)

awards_players['award_weight'] = awards_players['award'].map(award_weights)
awards_players['num_prizes'] = awards_players.groupby('playerID')['award_weight'].cumsum()
awards_players = awards_players.drop_duplicates(subset=['playerID', 'year'], keep='last')

awards_players = awards_players.drop(columns=['award', 'award_weight'], axis=1)

awards_players.to_csv('./Modified/modified_awards.csv', index=False)


### Players

- drop college, collegeOther, pos and lastseason
- remove who has height = 0
- replace height error (9) with the average
- replace weight error (0) with the average


In [ ]:
players['height'] = pd.to_numeric(players['height'], errors='coerce')
players = players.drop(columns=['college', 'pos', 'collegeOther', 'lastseason', 'firstseason'], axis=1)

players = players[players['height'] > 0]

# Calculate the mean of non-zero heights
mean_height = players[(players['height'] != 0) | (players['height'] != 9)]['height'].mean()
# Replace 9 with the mean height
players['height'] = players['height'].replace(9, mean_height)

# Calculate the mean of non-zero weights
mean_weight = players[players['weight'] != 0]['weight'].mean()
players['weight'] = players['weight'].replace(0, mean_weight)


### Players_teams

- statistics: info about the previous year
    - if never appeared before, null ( we doesnt know anything about that player at the beginning of the season)

In [ ]:
players_teams = players_teams.drop(columns=['lgID', 'PostGP','PostGS','PostMinutes','PostPoints','PostoRebounds','PostdRebounds','PostRebounds','PostAssists','PostSteals','PostBlocks','PostTurnovers','PostPF','PostfgAttempted','PostfgMade','PostftAttempted','PostftMade','PostthreeAttempted','PostthreeMade','PostDQ'], axis=1)
players_teams.sort_values(by=["playerID", "year", "stint"], inplace=True)
testing_set_with_stats = pd.DataFrame()

previous_columns = ['GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq']

players = players.rename(columns={'bioId': 'playerID'})

for year in range(2,12):

    players_in_year = players_teams[players_teams['year'] == year]

    players_in_year = players_teams[players_teams['playerID'].isin(players_in_year['playerID'])]

    players_in_year = players_in_year[players_in_year['year'] < year]

    last_season_per_player = players_in_year.groupby('playerID')['year'].max().reset_index()

    last_season_stats = players_teams.merge(last_season_per_player, on=['playerID', 'year'], how='right')

    last_year_sum = last_season_stats.groupby('playerID').sum()[previous_columns].reset_index()

    testing_set_for_year = last_year_sum.merge(players, on=['playerID'], how='left')
    
    testing_set_for_year['year'] = year

    testing_set_with_stats = pd.concat([testing_set_with_stats, testing_set_for_year], ignore_index=True)
    

players_teams= players_teams[['playerID', 'year', 'tmID']]

testing_set_with_stats = testing_set_with_stats.merge(players_teams, on=['playerID', 'year'], how='right')

players_teams = testing_set_with_stats



- add a column called Age, considering the year of the season. Calculated through the merge between two datasets~
- number of prizes of each player (according to the year)
- drop birthdate and deathdate


In [ ]:
def calculate_age(row):
    birth_date = row['birthDate']
    # Check if birth_date is not NaN (i.e., it's not a float)
    if not pd.isnull(birth_date):
        birth_year = int(str(birth_date)[:4])
        age = 2000 + row['year'] - birth_year
        return age
    else:
        # Handle NaN or float values in 'birthDate' column
        return None 


players_teams["birthDate"] = players_teams["playerID"].map(players.set_index("playerID")["birthDate"])

players_teams["height"] = players_teams["playerID"].map(players.set_index("playerID")["height"])

players_teams["weight"] = players_teams["playerID"].map(players.set_index("playerID")["weight"])

# Apply the function to each row
players_teams['age'] = players_teams.apply(calculate_age, axis=1)


merged_data = players_teams.merge(awards_players, on=['playerID', 'year'], how='left')


merged_data['num_prizes'] = merged_data.groupby('playerID')['num_prizes'].ffill()


merged_data['num_prizes'].fillna(0, inplace=True)


merged_data['num_prizes'] = merged_data.groupby('playerID')['num_prizes'].shift(1)


merged_data.fillna(0, inplace=True)


merged_data = merged_data.rename(columns={'num_prizes': 'players_prizes'})

merged_data = merged_data.drop('birthDate', axis=1)
merged_data = merged_data.drop('deathDate', axis=1)



### Coaches

- drop lgID, stint, post_wins and post_losses
- create column num_prizes (merging awards_players)

In [ ]:
coaches = coaches.drop(columns=['lgID'], axis=1)
#coaches = coaches.drop(columns=['stint'], axis=1)

coaches = coaches.merge(awards_players, left_on=['coachID', 'year'], right_on=['playerID', 'year'], how='left')
coaches.sort_values(by=["coachID", "year"], inplace=True)

coaches['num_prizes'] = coaches.groupby('coachID')['num_prizes'].ffill()

# Fill NaN values with 0 for players with no prizes 
coaches['num_prizes'].fillna(0, inplace=True)

coaches['num_prizes'] = coaches.groupby('coachID')['num_prizes'].shift(1)

coaches['num_prizes'].fillna(0, inplace=True)


coaches = coaches.drop(columns=['playerID'], axis=1)


In [ ]:
coaches.sort_values(by=["coachID", "year", "stint"], inplace=True)

testing_set_with_stats = pd.DataFrame()

previous_columns = ['won', 'lost', 'post_wins', 'post_losses']

for year in range(2,11):

    players_in_year = coaches[coaches['year'] == year]

    players_in_year = coaches[coaches['coachID'].isin(players_in_year['coachID'])]

    players_in_year = players_in_year[players_in_year['year'] < year]

    last_season_per_player = players_in_year.groupby('coachID')['year'].max().reset_index()

    last_season_stats = coaches.merge(last_season_per_player, on=['coachID', 'year'], how='right')
    last_season_stats['won'] = pd.to_numeric(last_season_stats['won'], errors='coerce')
    last_season_stats['lost'] = pd.to_numeric(last_season_stats['lost'], errors='coerce')
    last_season_stats['post_wins'] = pd.to_numeric(last_season_stats['post_wins'], errors='coerce')
    last_season_stats['post_losses'] = pd.to_numeric(last_season_stats['post_losses'], errors='coerce')

    last_year_sum = last_season_stats.groupby('coachID').sum()[previous_columns].reset_index()

    #testing_set_for_year = last_year_sum.merge(players, on='playerID', how='left')
    
    last_year_sum['year'] = year

    testing_set_with_stats = pd.concat([testing_set_with_stats, last_year_sum], ignore_index=True)
    

testing_set_with_stats = testing_set_with_stats.merge(coaches, on=['coachID', 'year'], how='right')

testing_set_with_stats = testing_set_with_stats.drop(columns=['won_y', 'lost_y', 'post_wins_y', 'post_losses_y'], axis=1)

testing_set_with_stats = testing_set_with_stats.dropna(thresh=6, axis=0)

coaches = testing_set_with_stats

coaches = coaches.rename(columns={'won_x': 'coach_won'})
coaches = coaches.rename(columns={'lost_x': 'coach_lost'})
coaches = coaches.rename(columns={'post_wins_x': 'coach_post_wins'})
coaches = coaches.rename(columns={'post_losses_x': 'coach_post_losses'})


coaches.to_csv('./Modified/modified_coaches.csv', index=False)

### Teams

- drop seeded, confID, divID, franchID, arena, lgID and name
- corrected error: one season it was registered two winners

In [ ]:
teams = teams.drop(columns=['seeded', 'divID', 'franchID', 'arena', 'lgID', 'name', 'tmORB'], axis=1)
teams = teams.drop(columns=['tmDRB'], axis=1)
teams = teams.drop(columns=['tmTRB'], axis=1)
teams = teams.drop(columns=['opptmORB'], axis=1)
teams = teams.drop(columns=['opptmDRB'], axis=1)
teams = teams.drop(columns=['opptmTRB'], axis=1)


stats_teams = ['o_fgm','o_fga','o_ftm','o_fta','o_3pm','o_3pa','o_oreb','o_dreb','o_reb','o_asts','o_pf','o_stl','o_to','o_blk','o_pts','d_fgm','d_fga','d_ftm','d_fta','d_3pm','d_3pa','d_oreb','d_dreb','d_reb','d_asts','d_pf','d_stl','d_to','d_blk','d_pts','GP','homeW','homeL','awayW','awayL','confW','confL','min']
teams = teams.drop(columns=stats_teams, axis=1)

columns_to_shift = ['won', 'lost', 'attend', 'rank']
columns_post = ['firstRound', 'semis', 'finals']

teams.sort_values(by=["tmID", "year"])


""" for col in columns_to_shift:
    teams[f'prev_{col}'] = teams.groupby('tmID')[col].shift(1)
    teams[f'prev_{col}'].fillna(0, inplace=True) """


for col in columns_post:
    teams[f'prev_{col}'] = label_encoder.fit_transform(teams.groupby('tmID')[col].shift(1))

teams = teams.drop(columns=columns_post, axis=1)

teams = teams.drop(columns=columns_to_shift, axis=1)


- calculate the number of prizes a team has (total of prizes the players have), creating the column num_prizes

In [ ]:
teams['year'] = pd.to_numeric(teams['year'], errors='coerce')
teams.sort_values(by=["tmID", "year"])

playerAndTeams = merged_data.merge(teams, on=["tmID", "year"])

# Group by team and year, summing the number of prizes
team_prizes = playerAndTeams.groupby(['tmID', 'year'])['players_prizes'].sum().reset_index()

# Sort the result by year
team_prizes.sort_values(by='year', inplace=True)

teams = teams.merge(team_prizes, on=["tmID", "year"])


- calculate the sum of the statistics of the players of each team

In [ ]:
# Define the columns to sum
#sum_columns = ['previous_GP', 'previous_GS', 'previous_minutes', 'previous_points', 'previous_oRebounds', 'previous_dRebounds', 'previous_rebounds', 'previous_assists', 'previous_steals', 'previous_blocks', 'previous_turnovers', 'previous_PF', 'previous_fgAttempted', 'previous_fgMade', 'previous_ftAttempted', 'previous_ftMade', 'previous_threeAttempted', 'previous_threeMade', 'previous_dq']

sum_columns = ['GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq']

# Sum the columns for each team and year
summed_columns = playerAndTeams.groupby(['tmID', 'year'])[sum_columns].sum().reset_index()

summed_columns.columns = ['tmID', 'year'] + ['sum_' + col for col in sum_columns]

# Merge the summed columns back into the 'teams' table
teams = pd.merge(teams, summed_columns, how='left', on=['tmID', 'year'])

- calculate the mean of the height, weight and age of the players of each team

In [ ]:

team_means = playerAndTeams.groupby(['tmID', 'year']).agg({
    'height': 'mean',
    'weight': 'mean',
    'age': 'mean',
}).reset_index()
teams = pd.merge(teams, team_means, on=['tmID', 'year'], how='left')

- add to the table teams the number of prizes the coach has
- number of victories and defeats of coach in previous year

In [ ]:
""" teams = pd.merge(teams, coaches[['tmID', 'year', 'coach_won', 'coach_lost']], on=['tmID', 'year'], how='left')

teams['coach_won'] = teams['coach_won'].fillna(0)
teams['coach_lost'] = teams['coach_lost'].fillna(0) """

merged_df = pd.merge(teams, coaches, on=['year', 'tmID'], how='left')

merged_df['num_prizes'] = merged_df['num_prizes'].fillna(0)

teams['coaches_prizes'] = merged_df['num_prizes']


In [ ]:
teams.to_csv('./Modified/modified_teams.csv', index=False)


- enconde categorical columns
- split data

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'playoff' column
label_encoder_playoff = LabelEncoder()
label_encoder_tmID = LabelEncoder()
label_encoder_confID = LabelEncoder()

teams['playoff'] = label_encoder_playoff.fit_transform(teams['playoff'])
teams['tmID'] = label_encoder_tmID.fit_transform(teams['tmID'])
teams['confID'] = label_encoder_confID.fit_transform(teams['confID'])


columns_to_normalize = [
    'sum_GP', 'sum_GS', 'sum_minutes', 'sum_points', 'sum_oRebounds',
    'sum_dRebounds', 'sum_rebounds', 'sum_assists', 'sum_steals', 'sum_blocks',
    'sum_turnovers', 'sum_PF', 'sum_fgAttempted', 'sum_fgMade', 'sum_ftAttempted',
    'sum_ftMade', 'sum_threeAttempted', 'sum_threeMade', 'sum_dq'
]

teams[columns_to_normalize] = (teams[columns_to_normalize] - teams[columns_to_normalize].mean()) / teams[columns_to_normalize].std()


train_data = teams[teams['year'].between(2, 9)]
#train_data = train_data[train_data['year'] > 1]
test_data = teams[teams['year'] == 10]

### Train and evaluate

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import time

# Use the same train_data and test_data from your code
X_train = train_data.drop(['playoff'], axis=1)
y_train = train_data['playoff']
X_test = test_data.drop(['playoff'], axis=1)
y_test = test_data['playoff']

# Define a list of classifiers
classifiers = [
    ("Logistic Regression", LogisticRegression(solver='liblinear', random_state=0), {'C': [0.001, 0.01, 0.1, 1, 10]}),
    ("Support Vector Machine", SVC(kernel='linear', random_state=0), {'C': [0.001, 0.01, 0.1, 1, 10]}),
    ("Decision Tree", DecisionTreeClassifier(random_state=0), {'max_depth': [None, 5, 10, 15]}),
    ("Random Forest", RandomForestClassifier(random_state=0), {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10]}),
    ("K-Nearest Neighbors", KNeighborsClassifier(), {'n_neighbors': [3, 5, 7]}),
    ("Naive Bayes", GaussianNB(), {})
]

# Create an empty dictionary to store performance metrics
performance_metrics = {}
roc_curve_data = []

# Initialize variables to calculate averages
average_accuracy = 0
average_roc_auc = 0

for name, clf, param_grid in classifiers:
    start_time = time.time()
    # Perform Grid Search
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(X_train, y_train)

    # Get the best model
    best_clf = grid_search.best_estimator_

    elapsed_time = time.time() - start_time

    # Use the best model to make predictions
    predictions = best_clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    average_accuracy += accuracy

    # Calculate ROC curve and AUC
    if hasattr(best_clf, "predict_proba"):
        y_score = best_clf.predict_proba(X_test)[:, 1]
    else:
        y_score = best_clf.decision_function(X_test)

    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    average_roc_auc += roc_auc

    # Store performance metrics in a dictionary
    performance_metrics[name] = {
        "Best Parameters": grid_search.best_params_,
        "Accuracy": accuracy,
        "ROC AUC": roc_auc,
        "Confusion Matrix": confusion_matrix(y_test, predictions),
        "Classification Report": classification_report(y_test, predictions),
        "ROC Curve": (fpr, tpr),
        "Elapsed Time": elapsed_time,
        "Model": best_clf
    }
    # Append ROC curve data for later averaging
    roc_curve_data.append((fpr, tpr))

# Calculate averages
average_accuracy /= len(classifiers)
average_roc_auc /= len(classifiers)

# Print the performance metrics and create ROC curve graphs
for name, metrics in performance_metrics.items():
    print(f"{name} Performance Metrics:")
    print(f"Best Parameters: {metrics['Best Parameters']}")
    print(f"Accuracy: {metrics['Accuracy']}")
    print(f"ROC AUC: {metrics['ROC AUC']}")
    print(f"Elapsed Time: {metrics['Elapsed Time']:.2f} seconds") 
    plt.figure(figsize=(8, 6))
    sns.heatmap(metrics['Confusion Matrix'], annot=True, fmt='d', cmap='Blues')
    plt.title(f'{name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    print("Classification Report:")
    print(metrics['Classification Report'])

    # Plot ROC Curve
    plt.figure()
    fpr, tpr = metrics["ROC Curve"]
    plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {metrics["ROC AUC"]:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{name} ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    
plt.figure()
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# Calculate and plot the average ROC curve
mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.mean([np.interp(mean_fpr, fpr, tpr) for fpr, tpr in roc_curve_data], axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='b', label=f'Average ROC Curve (AUC = {mean_auc:.2f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Average ROC Curve')
plt.legend(loc="lower right")
plt.show()

# Print averages
print(f"Average Accuracy: {average_accuracy:.2f}")
print(f"Average ROC AUC: {average_roc_auc:.2f}")


### Predict Year 11 (new year)

In [ ]:
# Iterate through all models
for name, metrics in performance_metrics.items():
    current_model = metrics["Model"]

    train_data = teams[teams['year'] != 11]
    test_data = teams[teams['year'] == 11]

    X_train = train_data.drop(['playoff'], axis=1)
    y_train = train_data['playoff']
    X_test = test_data.drop(['playoff'], axis=1)

    # Train the current model
    current_model.fit(X_train, y_train)

    # Make predictions using the current model
    predictions = current_model.predict(X_test)

    # Create a submission DataFrame for the current model
    submission_df = pd.DataFrame({
        'tmID': test_data['tmID'],
        'playoff': predictions
    })

    # Decode team IDs if necessary
    decoded_teams = label_encoder_tmID.inverse_transform(submission_df['tmID'])
    submission_df['tmID'] = decoded_teams

    # Save the submission file for the current model
    submission_df.to_csv(f'submission_{name}.csv', index=False)

### Feature importance

In [ ]:
logreg = LogisticRegression(solver='liblinear', random_state=0)
logreg.fit(X_train, y_train)

# Step 6: Make predictions
y_pred = logreg.predict(X_test)

inputs = teams.columns
feature_importance = abs(logreg.coef_[0])
feature_importance = sorted(zip(inputs, feature_importance), key=lambda x: x[1], reverse=True)

print("Feature importance:")
for i in range(len(feature_importance)):
    print(f"{feature_importance[i][0]}: {feature_importance[i][1]}")

In [ ]:
merged_data.to_csv('./Modified/players_playersTeams.csv', index=False)

### Correlation matrix

In [ ]:
# Create a correlation matrix for all columns
correlation_matrix = train_data.corr()

# Set the size of the heatmap
plt.figure(figsize=(10, 8))

# Create a heatmap using seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.3)

# Set the title
plt.title("Correlation Matrix")

# Show the heatmap
plt.show()

In [ ]:
""" teams_before = pd.read_csv('./Dataset/teams.csv')

print(teams_before.columns)
teams_before['playoff'] = label_encoder.fit_transform(teams_before['playoff'])
#teams['previous_playoff'] = label_encoder.fit_transform(teams['previous_playoff'])
teams_before['tmID'] = label_encoder.fit_transform(teams_before['tmID'])
teams_before['confID'] = label_encoder.fit_transform(teams_before['confID'])
teams_before['lgID'] = label_encoder.fit_transform(teams_before['lgID'])
teams_before['franchID'] = label_encoder.fit_transform(teams_before['franchID'])
teams_before['firstRound'] = label_encoder.fit_transform(teams_before['firstRound'])
teams_before['semis'] = label_encoder.fit_transform(teams_before['semis'])
teams_before['finals'] = label_encoder.fit_transform(teams_before['finals'])
teams_before['name'] = label_encoder.fit_transform(teams_before['name'])
teams_before['arena'] = label_encoder.fit_transform(teams_before['arena'])




train_data = teams_before[teams_before['year'] <= 9]
test_data = teams_before[teams_before['year'] == 10]

# Use the same train_data and test_data from your code
X_train = train_data.drop(['playoff'], axis=1)
y_train = train_data['playoff']
X_test = test_data.drop(['playoff'], axis=1)
y_test = test_data['playoff']

from sklearn.impute import SimpleImputer

# Initialize the imputer with a strategy (e.g., mean, median, most_frequent, constant)
imputer = SimpleImputer(strategy='mean')

# Fit and transform the imputer on your training data
X_train_imputed = imputer.fit_transform(X_train)

# Transform the imputer on your test data (using statistics learned from training data)
X_test_imputed = imputer.transform(X_test)

# Now, you can use X_train_imputed and X_test_imputed for training and testing your models

# Define a list of classifiers
classifiers = [
    ("Logistic Regression", LogisticRegression(solver='liblinear', random_state=0)),
    ("Decision Tree", DecisionTreeClassifier(random_state=0)),
    ("Random Forest", RandomForestClassifier(random_state=0)),
    ("Support Vector Machine", SVC(kernel='linear', random_state=0)),
    ("K-Nearest Neighbors", KNeighborsClassifier())
]

# Create an empty dictionary to store accuracies
accuracies = {}

# Iterate through each classifier, train, predict, and store accuracy
for name, clf in classifiers:
    clf.fit(X_train_imputed, y_train)
    predictions = clf.predict(X_test_imputed)
    accuracy = accuracy_score(y_test, predictions)
    accuracies[name] = accuracy

# Print the accuracies for each model
for name, accuracy in accuracies.items():
    print(f"{name} Accuracy: {accuracy}")
 """